## Run Imports

In [1]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
import warnings
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

# Ignore all warnings
warnings.filterwarnings('ignore')

# Read parquet file

In [ ]:
# Replace 'path/to/your/file.parquet' with the actual file path
parquet_file = 'path/to/your/file.parquet'

# Read the Parquet file
df_required = pd.read_parquet(parquet_file)


df_required

## Calculating aggregates

In [ ]:
groupedStats = df_required.groupby('TRIGGER_NAME')['CPU_TIME'].agg(['max','mean','std', 'var']).round(2).fillna(0)

groupedStats

# For standardization

In [151]:
# # Initialize the scaler
# scaler = StandardScaler()


# # Fit the scaler to the data and transform it
# scaled_features = scaler.fit_transform(df_required[['CPU_TIME']])

# # Convert the scaled features back into a DataFrame
# df_required['CPU_TIME'] = scaled_features

## Local Outlier Factor model

In [ ]:
model = LocalOutlierFactor(n_neighbors=350, contamination=0.2)
localOutlierFactor_model = model.fit_predict(groupedStats)
localOutlierFactor_model

## Add Anomalies

In [ ]:
groupedStats['ANOMALIES'] = localOutlierFactor_model
groupedStats

## LOF model anomalies

In [ ]:
groupedStats_req = groupedStats.loc[groupedStats['ANOMALIES'] == -1]
groupedStats_req

## Filter results based on max of CPU Time

In [ ]:
filteredStats = groupedStats_req[groupedStats_req['max'] > 3000]

## Output to CSV

In [22]:
# Save the DataFrame to 'output.csv'
filteredStats.to_csv('apex_triggers_local_outlier_factor.csv', index=True)
